# L12: Map Reduce && Fold
* This document is subject to change before Lecture

## Overview
* Anonymous Functions
* HOF
* Filter
* Map
* FoldLeft - Reduce
* Loops as HOFs
* Other Uses


## Anonymous Functions
* Scala allows us to create anonymous functions
* These are quite helpful if I am only going to use a function once
* Let’s look at some cases where it’s helpful and where it is less useful


### Example 1

In [3]:
val x0 = 2
println(x0 + 2 * 7 / 8)
val x1 = 10
println(x1 + 2 * 7 / 8)
val x2 = 20
println(x2 + 2 * 7 / 8)
val x3 = 225
println(x3 + 2 * 7 / 8)

3
11
21
226


x0: Int = 2
x1: Int = 10
x2: Int = 20
x3: Int = 225

* What do we think about the above code block?
* Is it easy to read?
* Could it be easier to read?
* It is just an opinion, but I think the bellow refactoring of the code is easier to read

In [4]:
def foo(x:Int):Unit = {
    println(x + 2 * 7 / 8)
}

foo(2)
foo(10)
foo(20)
foo(225)

3
11
21
226


defined function foo

* Example 1 uses a named function
* Functions are useful for taking a pattern of work and abstracting it to something more general

### Qs ???

### Example 2

In [6]:
sealed trait Maths
type Number = Double

case class Plus(m1:Maths, m2:Maths) extends Maths
case class Minus(m1:Maths, m2:Maths) extends Maths
case class Times(m1:Maths, m2:Maths) extends Maths
case class Div(m1:Maths, m2:Maths) extends Maths
case class Const(n1:Number) extends Maths

defined trait Maths
defined type Number
defined class Plus
defined class Times
defined class Div
defined class Minus
defined class Const

* The folowing code block is a straight forward implementation of eval that we've seen before

In [7]:
def eval0(e:Maths):Number = e match {
    case Const(n) => n
    case Plus(m1, m2) => eval0(m1) + eval0(m2)
    case Minus(m1, m2) => eval0(m1) - eval0(m2)
    case Times(m1, m2) => eval0(m1) * eval0(m2)
    case Div(m1,m2) => eval0(m1) / eval0(m2)
}

defined function eval0

* The folowing code block reimplements eval using an "apply" patter (I don't know the technical term)
* It also uses many named functions that are only used once
    * plus
    * minus
    * times
    * div

In [9]:
def eval1(e:Maths):Number = {
    
    def applyBinary(m1:Maths, m2:Maths)(f:(Number,Number) => Number):Number = {
        val n1 = eval1(m1)
        val n2 = eval1(m2)
        f(n1,n2)
    }
    
    e match {
        case Const(n) => n
        case Plus(m1, m2) => {
            def plus(n1:Number, n2:Number):Number = { n1 + n2 }
            applyBinary(m1,m2)(plus)
        }
        case Minus(m1, m2) => {
            def minus(n1:Number, n2:Number):Number = { n1 - n2 }
            applyBinary(m1,m2)(minus)
        }
        case Times(m1, m2) => {
            def times(n1:Number, n2:Number):Number = { n1 * n2 }
            applyBinary(m1,m2)(times)
        }
        case Div(m1, m2) => {
            def div(n1:Number, n2:Number):Number = { n1 / n2 }
            applyBinary(m1,m2)(div)
        }
     }
}

defined function eval1

* Now that one doesn't look pleasant to read
* I think that is because we have all these named functions that we only use once.
* Below I have reimplemented this yet again

In [11]:
def eval2(e:Maths):Number = {
    
    def applyBinary(m1:Maths, m2:Maths)(f:(Number,Number) => Number):Number = {
        val n1 = eval2(m1)
        val n2 = eval2(m2)
        f(n1,n2)
    }
    
    e match {
        case Const(n) => n
        case Plus(m1, m2) => applyBinary(m1,m2){(n1, n2) => n1 + n2}
        case Minus(m1, m2) => applyBinary(m1,m2){(n1, n2) => n1 - n2}
        case Times(m1, m2) => applyBinary(m1,m2){(n1, n2) => n1 * n2}
        case Div(m1, m2) => applyBinary(m1,m2){(n1, n2) => n1 / n2}
    }
}

defined function eval2

* I think that is a bit easier to read
* It's a bit easier to maintain than our initial implementation
* It uses a named functions:
    * eval2
    * applyBinary
* And it uses many anonymous functions
    *  (n1, n2) => n1 + n2
    *  (n1, n2) => n1 - n2
    *  (n1, n2) => n1 * n2
    *  (n1, n2) => n1 / n2
* Scala has one more more trick I'd like to show you... the '_'

In [12]:
def eval3(e:Maths):Number = {
    
    def applyBinary(m1:Maths, m2:Maths)(f:(Number,Number) => Number):Number = {
        val n1 = eval3(m1)
        val n2 = eval3(m2)
        f(n1,n2)
    }
    
    e match {
        case Const(n) => n
        case Plus(m1, m2) => applyBinary(m1,m2){ _ + _ }
        case Minus(m1, m2) => applyBinary(m1,m2){ _ - _ }
        case Times(m1, m2) => applyBinary(m1,m2){ _ * _ }
        case Div(m1, m2) => applyBinary(m1,m2){ _ / _ }
    }
}

defined function eval3

* I'll let you make assumptions about what the '_' is doing there, but you might want to go google it

* As we've seen in Example 2, anonymous functions can be quite useful
* In particular if we decompose a problem into parts
    * I had things of the form 'eval(m1) \$\$ eval(m2)'
    * I decomposed to:
        * An 'apply' function that takes 2 maths and a function about \$\$
        * And expressions that make use of the 'apply' function
* Anonymous functions help make code more legible if a function will only be used once
* When you are first learning HOFs:
    * you may not be comfortable using anonymous funcitons
        * I suppose that is fine
        * Start by writing a named function as I did in 'eval1'
        * Then anonymize the functions like 'eval2'
        * If you are feeling adventurous, try using the '_' as I did in 'eval3'

### Qs ???

## HOF

In many languages, the use of for-loops/while loops to iterate is replaced by operations on data structures such as `map`, `filter` and `fold`. In this lecture, we provide a brief overview with some examples. We show how many varieties of loops or equivalently recursion, can be systematically replaced by these operations.

The main idea behind functional programming is that `functions are first class objects`. What this means is that just like integers, strings, lists and other data, functions can be passed around. 
1. They can be used as arguments to other functions.
2. Functions can be created from inside a function and returned as a value.
3. You can have variables (vars) that can be assigned to functions.
4. and many more ideas that we will revisit later...

Gist:
* Until now in this class I’ve only really talked about “First class functions”
* But there are also “Higher ordered functions” (HOFs)
    * If a function returns a function as output, then it is an HOF
    * If a function takes another function as input, then it is an HOF
* These have so many practical applications...


## Filter

### NumList
* In homework 3 you implemented an HOF named filterByN for a NumList defined something like:
    * $$\begin{array}
\mathbf{NumList} & \rightarrow & \mathbf{EmptyList} \\
& | &  \mathbf{Cons(Double, NumList)} \\
\end{array}$$
* Let’s look at this further, see code bellow:

In [1]:
sealed trait NumList {
    def filter(f:(Double) => Boolean):NumList = this match {
        case EmptyList => EmptyList
        case Cons(h,t) => if (f(h)) { Cons(h, t filter f) } else { t filter f }
    }
}
case object EmptyList extends NumList
case class Cons(head:Double, tail:NumList) extends NumList


def filterByN(lst:NumList, f:(Double) => Boolean):NumList = lst match{
    case EmptyList => EmptyList
    case Cons(head, tail) => {
        if (f(head)) {
            Cons(head, filterByN(tail, f))
        } else {
            filterByN(tail, f)
        }
    }
}


//
// TESTS
//
val l0 = EmptyList  // []
val l1 = Cons(1, Cons(2, Cons(3, EmptyList)))  // [1,2,3]
val l2 = Cons(16, Cons(28, Cons(8, EmptyList)))  // [16, 28, 8]

val evenL0 = EmptyList  // []
val evenL1 = Cons(2, EmptyList)  // [2]
val evenL2 = Cons(16, Cons(28, Cons(8, EmptyList)))  // [16, 28, 8]

val isEven = (x:Double) => { x % 2 == 0 }
assert(filterByN(l0, isEven) == evenL0)
assert(filterByN(l1, isEven) == evenL1)
assert(filterByN(l2, isEven) == evenL2)
// Don't need the isEven val...
assert(filterByN(l2, (x:Double) => { x % 2 == 0 }) == evenL2)

val isOdd = (x:Double) => { x % 2 == 1 }
val oddL0 = EmptyList  // []
val oddL1 = Cons(1, Cons(3, EmptyList))  // [1,3]
val oddL2 = EmptyList  // []


assert((l0 filter isEven) == evenL0)
assert((l1 filter isEven) == evenL1)
assert((l2 filter isEven) == evenL2)
assert((l0 filter isOdd) == oddL0)
assert((l1 filter isOdd) == oddL1)
assert((l2 filter isOdd) == oddL2)

defined trait NumList
defined object EmptyList
defined class Cons
defined function filterByN
l0: EmptyList.type = EmptyList
l1: Cons = Cons(1.0, Cons(2.0, Cons(3.0, EmptyList)))
l2: Cons = Cons(16.0, Cons(28.0, Cons(8.0, EmptyList)))
evenL0: EmptyList.type = EmptyList
evenL1: Cons = Cons(2.0, EmptyList)
evenL2: Cons = Cons(16.0, Cons(28.0, Cons(8.0, EmptyList)))
isEven: Double => Boolean = ammonite.$sess.cmd0$Helper$$Lambda$3217/188875775@47803a68
isOdd: Double => Boolean = ammonite.$sess.cmd0$Helper$$Lambda$3219/658755917@5fc79ba7
oddL0: EmptyList.type = EmptyList
oddL1: Cons = Cons(1.0, Cons(3.0, EmptyList))
oddL2: EmptyList.type = EmptyList

### filter
* Scala has a type List[A]
* One of the methods of List[A] is a method called filter
* filter
    * operates on a List[A] 'l'
    * and a function from A to Boolean 'f'
    * and returns a List[A] 'lp
    * This will go through 'l' and observe each element 'li'
        * This will find 'bi' equal to f(li)
        * if 'bi' is true
            * then li is put into output lp
            * else it is not put in the output
* **NOTE: the nature of filter makes it a solid candidate for parallelization**
* filter might looks something like bellow:

~~~
sealed trait List[+A] {
	def filter[A](f:(A) => Boolean):List[A] = this match {
		case Nil => Nil
		case head :: tail => {
			if ( f(head) ) {
				head :: (tail filter f)
			} else {
				tail filter f
			}
		}
	}
}
~~~

* Note that this works on an abstract type “A”
    * It’s nothing special
    * It exists in other languages as well

* Here is how I can use filter method of List[A] in scala:




In [3]:
// here A is a double...
val lDoubles = List(1.0,2.0,3.0,4.0,5.0,6.0)
val lDoublesRelEven3:List[Double] = List(3.0,6.0)
val isRelEven3 = (d:Double) => (d%3) == 0
assert((lDoubles filter isRelEven3) == lDoublesRelEven3)

lDoubles: List[Double] = List(1.0, 2.0, 3.0, 4.0, 5.0, 6.0)
lDoublesRelEven3: List[Double] = List(3.0, 6.0)
isRelEven3: Double => Boolean = ammonite.$sess.cmd2$Helper$$Lambda$3382/1648454356@e308a9c

In [4]:
// here A is Char
val lChars = List('a','b','c','d','e','f','g')
val lVowels = List('a','e')
val lConst = List('b','c','d', 'f','g')
val isVowel = (c:Char) => Set('a','e','i','o','u') contains c
assert((lChars filter isVowel) == lVowels)
assert((lChars filter {!isVowel(_)}) == lConst)  // shows some fancy scala
// _ the all powerfull!

lChars: List[Char] = List('a', 'b', 'c', 'd', 'e', 'f', 'g')
lVowels: List[Char] = List('a', 'e')
lConst: List[Char] = List('b', 'c', 'd', 'f', 'g')
isVowel: Char => Boolean = ammonite.$sess.cmd3$Helper$$Lambda$3394/1549925612@627a7e72

* filter operates on a collection of data of length 'n'
* filter returns a collection of data of length 'm'
    * such that m <= n
* filter will not change any data in your collection

In [12]:
// Other examples???

## map
* Scala’s type List[A] also has a method 'map'
* Map
    * inputs:
        * l: List[A]
        * f: (A) => B
    * output: List[B]
    * let each element of l be denoted as 'li'
    * f(li) produces 'lip'
    * join each lip in order to construct an output lp
* **NOTE: the nature of map makes it a solid candidate for parallelization**
* it might look like bellow:

~~~
sealed trait List[+A] {
	def map[A,B](f:(A) => B):List[B] = this match {
		case Nil => Nil
		case head :: tail => f(head) :: (tail map f)
	}
}
~~~

* Here is how I can use it:




In [5]:
// here A is an Int
// here B is a Boolean
val lInts = List(5,8,13)
println(lInts map {(n:Int) => n % 2 == 0})
assert((lInts map {(n:Int) => n % 2 == 0}) == List(false, true, false))
assert((lInts map {_ % 2 == 0}) == List(false, true, false))

List(false, true, false)


lInts: List[Int] = List(5, 8, 13)

In [6]:
// here A is an Int
// here B is an Int
println(lInts map {(n:Int) => n + 1})
assert((lInts map {(n:Int) => n + 1}) == List(6,9,14))
assert((lInts map {_ + 1}) == List(6,9,14))

List(6, 9, 14)


* map operates on a collection of length 'n' and returns a collection of length 'n'
* map changes all the data in the collection by applying the same function to each datum of the collection

In [ ]:
// ??? more things using map

## foldLeft
* Scala’s type List[A] also has a method foldLeft
* A better name might be “scanFromLeftToRightAndAccumulateSomeValue”
    * But then I doubt anyone would use it…
* foldLeft
    * input:
        * l: List[A]
        * acci: B
        * f: (B,A) => B
    * output lp: B
    * scan the list from left to right observe some element of l 'li'
    * compute f(acci, li) and use that as 'acci' for the next 'li' observed
    
* **NOTE: foldLeft is not so easily parallellized as map and filter**
* **NOTE: map and filter can be built using foldLeft**
* it might look like bellow:

~~~
sealed trait List[+A] {
	def foldLeft[A,B](acc:B)(f:(B, A) => B):B = this match {
		case Nil => acc
		case head :: tail => {
			val accPrime = f(acc, head)
			tail.foldLeft(accPrime)(f)
		}
	}
}
~~~

* Here is how I can use it:




In [7]:
// Here A and B are both Lists[Int]
val sum = List(1,2,3).foldLeft(0){
    (acc, h) => {
        println(s"found list element $h")
        println(s"\tacc is: $acc")
        acc + h
    }
}

println(s"sum is: $sum")
assert(List(1,2,3).foldLeft(0){_ + _} == 6)  // #LoDashCan'tCompete

found list element 1
	acc is: 0
found list element 2
	acc is: 1
found list element 3
	acc is: 3
sum is: 6


sum: Int = 6

In [8]:
// Here A is Int and B is String
def formatList(l:List[Int]):String = {
    l.foldLeft(""){
        case ("", h) => s"[ $h"
        case (acc, h) => s"$acc, $h"
    } + " ]"
}

println(formatList(List(1,2,3)))

[ 1, 2, 3 ]


defined function formatList

* foldLeft operates on a collection of lenth 'n' and can return anything
* it can be used to implement 'filter' and 'map'

In [13]:
// filter
def foldFilter[A](l:List[A])(f:A => Boolean):List[A] = {
    // implement filter using foldLeft
    ???
}

defined function foldFilter

In [14]:
// map
def foldMap[A,B](l:List[A])(f:A => B): List[B] = {
    // implement map using foldLeft
    ???
}

defined function foldMap

In [15]:
// exists
/*
    search over collection List[A]
    for each datum:A observed
        if f(datum)
            then: return true
            else: continue
    return false
 */
def foldExists[A](l:List[A])(f:A => Boolean):Boolean = {
    // implement exists using foldLeft
    ???
}

defined function foldExists

## Other Uses
* These HOFs can replace loops
* These HOFs are constructs that can easily be applied to other types
* These HOFs are constructs that can easily be made in other languages.

### Loops as HOFs
* consider the python statement:
~~~
x = [5, 10, 2]
i = 0
l = len(x)
product = 0
while i < l:
    x_i = x[i]
    print(x_i)
    product = product * x_i
    i = i + 1
print(product)
~~~
* What is it's value?
    * ???
* Can we write it in Scala without a loop?

In [1]:
val x:List[Int] = List(5, 10, 2)
???

: 

### Trees
* Let's look at "map" on
    * a binary tree
    * a binned tree

#### Binary Tree

In [22]:
sealed abstract class Tree[+A]
case object EmptyTree extends Tree
case class Node[A](left:Tree[A], data:A, right:Tree[A]) extends Tree[A]

// ideally this is a method...
// but that is hard because of all the As up there...
// sorry...
// mapt will apply 'f' to each data point 'd' in tree 't'
def mapt[A,B](t:Tree[A])(f:(A) => B):Tree[B] = ???

defined class Tree
defined object EmptyTree
defined class Node
defined function mapt

In [16]:
val t0:Tree[Int] = Node(Node(EmptyTree, 1, EmptyTree), 6, EmptyTree)
val t0PlusPlus:Tree[Int] = Node(Node(EmptyTree, 2, EmptyTree), 7, EmptyTree)

assert(t0PlusPlus == mapt(t0){(d:Int) => d + 1})
assert(t0PlusPlus == mapt(t0){ _ + 1 })

t0: Tree[Int] = Node(Node(EmptyTree, 1, EmptyTree), 6, EmptyTree)
t0PlusPlus: Tree[Int] = Node(Node(EmptyTree, 2, EmptyTree), 7, EmptyTree)

In [ ]:
// More with Tree[A] ???

#### Binned Tree
* The binned binary search tree is a practical data structure
    * It has applications in Databases
    * Involves datasets where duplicates are important
* In practice, it is a balanced binary search tree which stores non-empty repetative lists of data at each node in the tree
* example:
    * $$\begin{array} &
Tree\ starts\ as & : & EmptyTree \\
add\ a\ 5 & : & Node(EmptyTree, \textbf{List(5)}, EmptyTree) \\
add\ a\ 2 & : & Node(Node(EmptyTree,\textbf{List(2)},EmptyTree), \textbf{List(5)}, EmptyTree) \\
add\ another\ 5 & : & Node(Node(EmptyTree,\textbf{List(2)},EmptyTree), \textbf{List(5, 5)}, EmptyTree) \\
add\ another\ 5 & : & Node(Node(EmptyTree,\textbf{List(2)},EmptyTree), \textbf{List(5, 5, 5)}, EmptyTree) \\
remove\ a\ 2 & : & Node(EmptyTree, \textbf{List(5, 5, 5)}, EmptyTree) \\
remove\ a\ 5 & : & Node(EmptyTree, \textbf{List(5, 5)}, EmptyTree) \\
\end{array} $$


In [18]:
type BinnedTree[A] = Tree[List[A]]

// map on bt shall apply f to every element in every list in bt
def mapbt[A,B](bt:BinnedTree[A])(f:(A) => B):BinnedTree[B] = ???

defined type BinnedTree
defined function mapbt

In [20]:
val bt0:BinnedTree[Int] = Node(Node(EmptyTree, List(1,1,1),EmptyTree), List(5,10,25), EmptyTree)
val bt0PlusPlus:BinnedTree[Int] = Node(Node(EmptyTree, List(2,2,2),EmptyTree), List(6,11,26), EmptyTree)

assert(bt0PlusPlus == mapbt(bt0){ _ + 1 })

bt0: BinnedTree[Int] = Node(
  Node(EmptyTree, List(1, 1, 1), EmptyTree),
  List(5, 10, 25),
  EmptyTree
)
bt0PlusPlus: BinnedTree[Int] = Node(
  Node(EmptyTree, List(2, 2, 2), EmptyTree),
  List(6, 11, 26),
  EmptyTree
)

### Python
* How do I do this magical thing in Python ?!
* See code...

## FoldRight
* similar to foldLeft, but it will scan the list from Right to Left then accumulate something
* it might look like:
~~~
sealed trait List[+A] {
    def foldRight[A,B](acc:A)(f:(A,B) => B) B = this match {
        case EmptyListy => acc
        case h :: t => {
            val accp = t.foldRight(acc)(f)
            f(h,accp)
        }
    }
}
~~~
* Note that the parameter 'f' looks a bit different from foldLeft

In [3]:
val lst = List(1,2,3)
val lstProduct = 6

println(s"The List is : $lst")

println("folding Left")
val productLeft = lst.foldLeft(1){
    (acc, h) => {  // Note this line
        println(s"\tFound element $h")
        acc * h
    }
}

println("folding Right")
val productRight = lst.foldRight(1){
    (h, acc) => {  // Note this line
        println(s"\tFound element $h")
        acc * h
    }
}

assert(lstProduct == productLeft)
assert(lstProduct == productRight)
assert(lstProduct == lst.foldLeft(1){ _ * _ })
assert(lstProduct == lst.foldRight(1){ _ * _ })

The List is : List(1, 2, 3)
folding Left
	Found element 1
	Found element 2
	Found element 3
folding Right
	Found element 3
	Found element 2
	Found element 1


lst: List[Int] = List(1, 2, 3)
lstProduct: Int = 6
productLeft: Int = 6
productRight: Int = 6

## Practice Problems

In [ ]:
// given a list, reverse the list

// ??? without an HOF

// ??? with an HOF

In [ ]:
// find the count of even numbers in a list

// ??? without an HOF

// ??? with an HOF

In [3]:
// does the list contain an odd number?

// ??? without an HOF

// ??? with an HOF

## Overview
* HOF
* Filter
* Map
* FoldLeft - Reduce
* Loops as HOFs
* Other Uses

## TODOs
* Homework and Quiz 4 is due Friday 02/22
* Homework 5 is due Sunday 02/24 - No Moodle Quiz!
* Project 1 is due Monday 03/04
* If you have questions about your spot exam, please let us know - in person or over Piazza
    * Some time in Recitation this Friday should be dedicated to reviewing the first spot exam.